In [3]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.datasets import cifar10
from keras.callbacks import LearningRateScheduler
from keras.models import model_from_json
from keras.models import load_model
from keras.losses import KLDivergence
from sklearn.metrics import accuracy_score
import numpy as np
import keras
from keract import get_activations
import math
import time
import matplotlib.pyplot as plt
import sys
import tensorflow as tf

import torch.nn.functional as F

#from wrnet_model import create_wide_residual_network
from prova_wrn import create_wide_residual_network
'''
Function that returns the trainand test data of the CIFAR10 already preprocessed
'''
def getCIFAR10():
    # input image dimensions
    img_rows, img_cols = 32, 32
    num_classes = 10

    # the data, split between train and test sets
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    
    # format of the tensor
    if K.image_data_format() == 'channels_first':
        x_train = x_train.reshape(x_train.shape[0], 3, img_rows, img_cols)
        x_test = x_test.reshape(x_test.shape[0], 3, img_rows, img_cols)
        input_shape = (3, img_rows, img_cols)
    else:
        x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)
        x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 3)
        input_shape = (img_rows, img_cols, 3)

    # convert in to float the images
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    
    # new normalization with z-score
    mean = np.mean(x_train,axis=(0,1,2,3))
    std = np.std(x_train,axis=(0,1,2,3))
    x_train = (x_train-mean)/(std+1e-7)
    x_test = (x_test-mean)/(std+1e-7)
    
    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    
    print('CIFAR10 loaded')
    return x_train,y_train,x_test,y_test

'''
Small function that returns the shape of the CIFAR10 images
'''
def getCIFAR10InputShape():
    img_rows, img_cols = 32, 32
    if K.image_data_format() == 'channels_first':
        input_shape = (3, img_rows, img_cols)
    else:
        input_shape = (img_rows, img_cols, 3)
        
    return input_shape

'''
Function that loads from a file the teacher
'''
def getTeacher(file_name):
    # Model reconstruction from JSON file
    with open(file_name + '.json', 'r') as f:
        model = model_from_json(f.read())

    # Load weights into the new model
    model.load_weights(file_name + '.h5')
    
    print('Teacher loaded from ' + file_name + '.h5')
    return model
    
'''
Function that loads from a file the teacher and test it on the CIRAF10 dataset
'''
def testTeacher(file_name):
    
    x_train,y_train,x_test,y_test = getCIFAR10()
    
    model = getTeacher(file_name)
    
    # define optimizer
    opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=opt_rms,
                  metrics=['accuracy'])

    # final evaluation on test
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Teacher test loss:', score[0])
    print('Teacher test accuracy:', score[1])

    
'''
Function that returns a simple student done by 2 convolutions, a maxpool and a final two fully connected layers
'''
def getStudent(input_shape):
    num_classes = 10
    model1,model2=create_wide_residual_network(input_shape, num_classes, N=2, k=1, dropout=0.)
    
    print('Student loaded')
    return model1,model2
    
'''
Function to try to train the simple sutdent in order to unerstand its capabilites
'''
def trainStudent(epochs):
    
    x_train,y_train,x_test,y_test = getCIFAR10()
    
    input_shape = getCIFAR10InputShape()
    
    model = getStudent(input_shape)
    
    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
    
    batch_size = 128
    n_batches = math.floor( x_train.shape[0] / batch_size)

    for e in range(epochs):
    
        for i in range(0,n_batches):
            imgs = x_train[i*batch_size:(i+1)*batch_size]
            labels = y_train[i*batch_size:(i+1)*batch_size]
            loss = model.train_on_batch(imgs,labels)
            print("Epoch: " + str(e+1) + " batch " + str(i) + " loss: " + str(loss[0]) + " acc: " + str( 100*loss[1]))
            
        score = model.evaluate(x_test, y_test, verbose=0)
        print('After epoch ' + str(e+1) + ' test loss ' + str(score[0]) + ' test accuracy ' + str(score[1]))

'''
Function that returns a simple generator
'''
def getGenerator():

        noise_shape = (100,)

        model = Sequential()
        
        img_shape = getCIFAR10InputShape()

        model.add(Dense(128*8**2, input_shape=noise_shape))
        model.add(Reshape((8, 8, 128)))
        model.add(BatchNormalization())
        
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2)) 
                  
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=(3,3), strides=1, padding="same"))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        
        model.add(Conv2D(3, kernel_size=(3,3), strides=1, padding="same"))
        model.add(BatchNormalization())   
        
        #model.summary()
        print('Generator loaded')
        return model

def mypositiveloss(y_true,y_pred):
    
    #y_true = K.log(y_true)
    #y_pred = K.log(y_pred)
    
    #loss=  keras.losses.kullback_leibler_divergence(y_true, y_pred)
    loss= tf.keras.losses.KLD(y_true, y_pred)
    
    return loss


def mynegativeloss(y_true,y_pred):
    
    #y_true = K.log(y_true)
    #y_pred = K.log(y_pred)
    
    #loss= keras.losses.kullback_leibler_divergence(y_true, y_pred)
    loss= tf.keras.losses.KLD(y_true, y_pred)
    gen_loss = -loss
    
    return gen_loss

def mypositiveswappedloss(y_true,y_pred):
    
    #y_true = K.log(y_true)
    #y_pred = K.log(y_pred)
    
    #loss= keras.losses.kullback_leibler_divergence(y_true, y_pred)
    loss= tf.keras.losses.KLD(y_pred, y_true)
    gen_loss = -loss
    
    return gen_loss
        
    
def main():
    
    x_train, y_train, x_test, y_test = getCIFAR10()
    
    teacher = getTeacher('model-16-2')
    teacher.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer='adam',
                  metrics=['accuracy'])
    
    input_shape = getCIFAR10InputShape()
    
    student_train, student_test = getStudent(input_shape)
    
    optim_stud = Adam(lr=2e-3, clipnorm=5.0)
    optim_gen = Adam(lr=1e-3, clipnorm=5.0)
    
    student_train.compile(loss=mypositiveloss ,
                optimizer=optim_stud)
    
    student_test.compile(loss="categorical_crossentropy",
                  optimizer='adam',
                  metrics=['accuracy'])
    
    generator = getGenerator()
    #generator.compile(loss=mynegativeloss, optimizer=optim_gen)
    
    student_train.trainable=False
    teacher.trainable = False

    z = Input(shape=(100,))

    #Generator makes a prediction
    fake_imgs = generator(z)

    #Discriminator attempts to categorise prediction
    y = student_train(fake_imgs)

    gan = Model(z, y)
    gan.compile(loss=mynegativeloss,optimizer=optim_gen)
    
    t = teacher(fake_imgs)
    gan2 = Model(z,t)
    gan2.compile(loss=mypositiveswappedloss,optimizer=optim_gen)
    
    print('Student Summary:')
    student_train.summary()
    print('Generator Summary:')
    generator.summary()
    print('GAN Summary:')
    gan.summary()
    
    n_batches = 152
    batch_size = 128
    log_freq = 30
    ns = 10
    
    for i in range(n_batches):
        
        noise = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = generator.predict(noise)
        
        t_predictions = teacher.predict(gen_imgs)
        g_loss = gan.train_on_batch(noise,t_predictions)
        s_predictions = student_train.predict(gen_imgs)
        g_loss2 = gan2.train_on_batch(noise,s_predictions)
        print('gloss2: ' + str(g_loss2))
        
        
        noise = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = generator.predict(noise)
        
        t_predictions = teacher.predict(gen_imgs)
        
        
        s_loss = 0
        for j in range(ns):
            noise = np.random.normal(0, 1, (batch_size, 100))
            gen_imgs = generator.predict(noise)
            s_loss += student_train.train_on_batch(gen_imgs,t_predictions)
        
        print('batch ' + str(i) + '/' + str(n_batches) + ' G loss: ' + str(g_loss) + ' S loss: ' + str(s_loss/ns))
        
        if (i % log_freq) == 0:
            score = student_test.evaluate(x_test, y_test, verbose=0)
            print('Student test loss: '  + str(score))
            
            model_json = student_test.to_json()
            with open('tmp-model' + str(i) + '.json','w') as json_file:
                json_file.write(model_json)
            student_test.save_weights('tmp-model' + str(i) + '.h5')
            print('saved model ' + str(i))
   
        
        
    score = student_test.evaluate(x_test, y_test, verbose=1)
    print('Final student test loss: '  + str(score))

    '''
    for i in range(0,n_batches):
        print('batch ' + str(i))
        imgs = x_train[i*batch_size:(i+1)*batch_size]
        labels = y_train[i*batch_size:(i+1)*batch_size]
        t_predictions = teacher.predict(imgs)
        s_predictions = student.predict(imgs)
        print('teacher predictions: ')
        print(t_predictions)
        print('student predictions: ')
        print(s_predictions)
        
        kl_div = tf.keras.losses.KLDivergence()
        
        
        # to print the KL divergence
        for j in range(batch_size):
            loss1 = kl_div(t_predictions[j],s_predictions[j])
            loss2 = tf.keras.losses.categorical_crossentropy(t_predictions[j],s_predictions[j])
            with tf.Session() as sess:
                init = tf.global_variables_initializer()
                sess.run(init)
                print('sample ' + str(j) + ' kl div: ' + str(loss1.eval()) + ' cat loss: ' + str(loss2.eval()))
        '''


main()


CIFAR10 loaded
Teacher loaded from model-16-2.h5


/home/test/advanceddeep/our_code/prova_wrn.py:38: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), padding="same", kernel_initializer="he_normal", use_bias=False, kernel_regularizer=<keras.reg...)`
  use_bias=False)(input)
/home/test/advanceddeep/our_code/prova_wrn.py:50: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), padding="same", strides=(1, 1), kernel_initializer="he_normal", use_bias=False, kernel_regularizer=<keras.reg...)`
  use_bias=False)(init)
/home/test/advanceddeep/our_code/prova_wrn.py:59: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), padding="same", kernel_initializer="he_normal", use_bias=False, kernel_regularizer=<keras.reg...)`
  use_bias=False)(x)
/home/test/advanceddeep/our_code/prova_wrn.py:63: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (1, 1), padding="same", strides=(1, 1), kernel_initializer="he_normal", use_bias=False, kernel_regularizer=<k

Wide Residual Network-16-1 created.
Student loaded
Generator loaded
Student Summary:
Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_39 (Conv2D)              (None, 32, 32, 16)   432         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_35 (BatchNo (None, 32, 32, 16)   64          conv2d_39[0][0]                  
__________________________________________________________________________________________________
activation_27 (Activation)      (None, 32, 32, 16)   0           batch_normalization_35[0][0]     
_______

gloss2: -1.1277186


/home/test/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/test/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


batch 0/152 G loss: -0.521895 S loss: 1.5802241444587708
Student test loss: [2.716512268447876, 0.10000000149011612]
saved model 0
gloss2: -1.0474535
batch 1/152 G loss: -0.6248786 S loss: 1.3776566147804261
gloss2: -1.1767131
batch 2/152 G loss: -0.66640866 S loss: 1.225106656551361
gloss2: -1.1019926
batch 3/152 G loss: -0.69629824 S loss: 1.0899369418621063
gloss2: -1.1516492
batch 4/152 G loss: -0.67080986 S loss: 0.9448484778404236
gloss2: -1.1864557
batch 5/152 G loss: -0.59 S loss: 0.7688574969768525
gloss2: -1.3190494
batch 6/152 G loss: -0.4519914 S loss: 0.6440325140953064
gloss2: -1.4337597
batch 7/152 G loss: -0.312952 S loss: 0.46653313040733335
gloss2: -1.619076
batch 8/152 G loss: -0.24739994 S loss: 0.4178831547498703
gloss2: -1.723657
batch 9/152 G loss: -0.23404987 S loss: 0.37992464005947113
gloss2: -1.8412712
batch 10/152 G loss: -0.31976604 S loss: 0.37207024693489077
gloss2: -1.8722763
batch 11/152 G loss: -0.32432532 S loss: 0.5094150960445404
gloss2: -1.7940148


batch 101/152 G loss: -0.51176447 S loss: 0.4961235076189041
gloss2: -2.2898006
batch 102/152 G loss: -0.5588675 S loss: 0.4615097165107727
gloss2: -2.238881
batch 103/152 G loss: -0.5038663 S loss: 0.46640937924385073
gloss2: -2.257115
batch 104/152 G loss: -0.54409766 S loss: 0.4612878829240799
gloss2: -2.1784708
batch 105/152 G loss: -0.49512932 S loss: 0.5353535711765289
gloss2: -2.21261
batch 106/152 G loss: -0.46328935 S loss: 0.5041564047336579
gloss2: -2.359419
batch 107/152 G loss: -0.48107728 S loss: 0.5437717437744141
gloss2: -2.1127942
batch 108/152 G loss: -0.51427037 S loss: 0.5993355870246887
gloss2: -2.1441705
batch 109/152 G loss: -0.5050767 S loss: 0.5031355917453766
gloss2: -2.1718953
batch 110/152 G loss: -0.5230468 S loss: 0.50323965549469
gloss2: -2.1776166
batch 111/152 G loss: -0.59704185 S loss: 0.4882546901702881
gloss2: -2.1621323
batch 112/152 G loss: -0.5856274 S loss: 0.5810249149799347
gloss2: -2.1695182
batch 113/152 G loss: -0.5446552 S loss: 0.60158167

In [12]:
# load the model we just trained
# if i use the last for CIFAR10
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.datasets import cifar10
from keras.callbacks import LearningRateScheduler
from keras.models import model_from_json
from keras.models import load_model
import numpy as np

batch_size = 128
num_classes = 10
epochs = 6

# input image dimensions
img_rows, img_cols = 32, 32

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 3, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 3, img_rows, img_cols)
    input_shape = (3, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_rows, img_cols, 3)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#old normalization
#x_train /= 255
#x_test /= 255

# new normalization with z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print('loading tmp-model0')
# Model reconstruction from JSON file
with open('tmp-model0.json', 'r') as f:
    model = model_from_json(f.read())

# Load weights into the new model
model.load_weights('tmp-model0.h5')

# define optimizer
opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt_rms,
              metrics=['accuracy'])

# final evaluation on test
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


print('loading tmp-mode150')
# Model reconstruction from JSON file
with open('tmp-model150.json', 'r') as f:
    model = model_from_json(f.read())

# Load weights into the new model
model.load_weights('tmp-model150.h5')

# define optimizer
opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt_rms,
              metrics=['accuracy'])

# final evaluation on test
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
loading tmp-model0
Test loss: 2.7111117263793947
Test accuracy: 0.10000000149011612
loading tmp-mode150
Test loss: 3.3176475238800047
Test accuracy: 0.10000000149011612


In [24]:
# try to negate a tensor
import tensorflow as tf
from tensorflow.keras import backend as K

def myKLDiv(a,b):
    division = tf.math.divide(soft_a,soft_b)
    log_div = K.log(division)
    res = K.sum(log_div)
    
    return res


a = K.random_uniform(shape=(1,3))
b = K.random_uniform(shape=(1,3))

soft_a = tf.nn.softmax(a)
soft_b = tf.nn.softmax(b)

log_a = K.log(soft_a)
log_b = K.log(soft_b)
    
loss= keras.losses.kullback_leibler_divergence(log_a, log_b)

loss_soft= keras.losses.kullback_leibler_divergence(soft_a, soft_b)

neg_loss_soft = -loss_soft

neg_loss = -loss

mykl = myKLDiv(soft_a,soft_b)

print(loss)

with tf.Session() as sess:
                init = tf.global_variables_initializer()
                sess.run(init)
                print('a: ' + str(a.eval()))
                print('b: ' + str(b.eval()))
                
                print('soft_a: ' + str(soft_a.eval()))
                print('soft_b: ' + str(soft_b.eval()))
                
                print('log_a: ' + str(log_a.eval()))
                print('log_b: ' + str(log_b.eval()))
                
                print('loss_soft ' + str(loss_soft.eval()))
                print('neg_loss_soft ' + str(neg_loss_soft.eval()))
                
                print('loss_log: ' + str(loss.eval()))
                print('neg loss_log: ' + str(neg_loss.eval()))
                
                print('my loss: ' + str(mykl.eval()))
                


Tensor("Sum_12:0", shape=(1,), dtype=float32)
a: [[0.06661725 0.8389952  0.23997462]]
b: [[0.9655373  0.6230657  0.59269714]]
soft_a: [[0.27257267 0.32507604 0.40235132]]
soft_b: [[0.2674539  0.42506498 0.3074811 ]]
log_a: [[-0.66898495 -1.4593475  -1.3649876 ]]
log_b: [[-1.4161048  -1.35178    -0.69602454]]
loss_soft [0.00532334]
neg_loss_soft [-0.01719323]
loss_log: [0.]
neg loss_log: [-0.]
my loss: -0.044802185


In [22]:
# design our custom KL loss

# KL(a,b) = SUM( a * log( a / b))

from keras.layers import Lambda
import tensorflow as tf
from tensorflow.keras import backend as K

a = K.random_uniform(shape=(1,3))
b = K.random_uniform(shape=(1,3))

soft_a = tf.nn.softmax(a)
soft_b = tf.nn.softmax(b)


division = tf.math.divide(soft_a,soft_b)
#division2 = divResult = Lambda(lambda x: x[0]/x[1])([soft_a,soft_b])
log_div = K.log(division)
res = K.sum(log_div)

with tf.Session() as sess:
                init = tf.global_variables_initializer()
                sess.run(init)
                print('a: ' + str(a.eval()))
                print('b: ' + str(b.eval()))
                
                print('soft_a: ' + str(soft_a.eval()))
                print('soft_b: ' + str(soft_b.eval()))
                
                print('div: ' + str(division.eval()))
                print('div2: ' + str(division2.eval()))
                print('log_div: ' + str(log_div.eval()))
                print('res: ' + str(res.eval()))


a: [[0.40674496 0.6270155  0.20576179]]
b: [[0.82592905 0.99604785 0.34396696]]
soft_a: [[0.3309812  0.2238779  0.44514093]]
soft_b: [[0.4950692  0.24711792 0.2578129 ]]
div: [[0.69207877 0.8602267  1.6768394 ]]
div2: [[1.1801109 1.2576282 0.6115273]]
log_div: [[ 0.6486425   0.02256223 -0.8076298 ]]
res: 0.083834425


In [ ]:


# pseudocodice per il paper

# for i in range(batches):
    # z = noise(100)
    # generated_images = Generator(z)
    # (output, teacher_activations) = teacher(generated_images)
    ''' Teacher is the pre-trained network that outputs its activations and the result 
    or it can outputs only the result and we can get the activations with K.function '''
    # combined.train_on_batch(generated_images, (outputs,teacher_activations)) 
    '''Combined is a network that has at the start the generator and then the freezed student.
    The labels it gets are the results and the activations of the teacher
    The loss is to increase the distance between the labels and the output of itself
    therefore this network has to output its intermediate activations'''
    
    # for j in range(ns):
        # student.train_on_batch(generated_images, (outputs,teacher_activations))
        ''' Student network that outputs its results and its intermediate activations, 
        and its loss is to match the output and activations of the teacher'''

    ''' if i don't find a way to output intermediate activations from a model we could always
        get the activations with K.fuction and the data, and then input them to the loss as the label.
        This method does not seems to increase too much the training time...
    '''


